In [34]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib

In [4]:
df_train = pd.read_excel("CYP2C9_dataset.xls", sheet_name="Training set")

In [5]:
df_train

,SMILES,Sp,nBM,ARR,nPyrimidines,HyWi_B(m),GATS2i,Eta_betaP_A,nRNR2,F01[C-N],MLOGP,Class
0,BrC(c1cccc2ccccc12)c3cccc4ccccc34,28.444,22,0.880,0,4.391,0.953,0.909,0,0,6.100,Active
1,Br\C(=C/c1ccccc1)\C=N\NC(=O)CNc2cccc3ccccc23,31.915,20,0.607,0,4.495,1.126,0.808,0,4,3.972,Active
2,Br\C(=C/c1ccccc1)\C=N\NC(=O)Cc2cccs2,25.034,14,0.524,0,4.356,1.080,0.725,0,2,2.974,Active
3,Br\C=C/1\CC[C@@H](C(=O)O1)c2cccc3ccccc23,23.591,13,0.524,0,4.282,1.050,0.658,0,0,3.722,Inactive
4,Brc1cc2OCOc2cc1\C=N\Nc3nc(nc(n3)N4CCOCC4)N5CCCC5,34.847,13,0.353,0,4.589,0.798,0.533,0,14,3.363,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
6380,o1c(\C=C\c2ccccc2)nnc1c3cccs3,21.159,17,0.800,0,3.923,1.070,0.861,0,2,4.100,Active
6381,o1c(cnc1c2ccc(cc2)c3oc(cn3)c4ccccc4)c5ccccc5,32.250,28,0.875,0,4.290,1.042,0.929,0,4,4.967,Inactive
6382,o1ccc(c1)c2ccc3ncnc(c3c2)n4ccnc4,21.762,21,0.913,1,3.976,0.772,0.900,0,9,2.417,Inactive
6383,o1ccc(c1)c2nc(c3ccccc3n2)n4ccnc4,21.762,21,0.913,1,3.976,0.934,0.900,0,9,2.685,Inactive


In [10]:
X_train = df_train.drop(["SMILES", "Class"], axis = 1)

In [19]:
X_train

,Sp,nBM,ARR,nPyrimidines,HyWi_B(m),GATS2i,Eta_betaP_A,nRNR2,F01[C-N],MLOGP
0,28.444,22,0.880,0,4.391,0.953,0.909,0,0,6.100
1,31.915,20,0.607,0,4.495,1.126,0.808,0,4,3.972
2,25.034,14,0.524,0,4.356,1.080,0.725,0,2,2.974
3,23.591,13,0.524,0,4.282,1.050,0.658,0,0,3.722
4,34.847,13,0.353,0,4.589,0.798,0.533,0,14,3.363
...,...,...,...,...,...,...,...,...,...,...
6380,21.159,17,0.800,0,3.923,1.070,0.861,0,2,4.100
6381,32.250,28,0.875,0,4.290,1.042,0.929,0,4,4.967
6382,21.762,21,0.913,1,3.976,0.772,0.900,0,9,2.417
6383,21.762,21,0.913,1,3.976,0.934,0.900,0,9,2.685


In [14]:
y_train = df_train["Class"].map({"Active": 1, "Inactive": 0}).values

In [15]:
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [6]:
df_test = pd.read_excel("CYP2C9_dataset.xls", sheet_name="Test set")

In [7]:
df_test

,SMILES,Sp,nBM,ARR,nPyrimidines,HyWi_B(m),GATS2i,Eta_betaP_A,nRNR2,F01[C-N],MLOGP,Class
0,BrC1=CC(=C)[C@@H]2O[C@@H]2C13OCCCO3,17.284,2,0.000,0,4.076,1.015,0.214,0,0,1.373,Inactive
1,Brc1ccc(Cc2cnc(NC(=O)CSc3nnc(c4ccncc4)n3CC=C)s...,38.466,24,0.629,0,4.723,1.118,0.734,0,11,4.643,Inactive
2,Brc1ccc(NC(=S)NNC(=O)CSc2nc3ccccc3s2)cc1,30.580,18,0.593,0,4.592,1.212,0.700,0,6,3.146,Active
3,Brc1ccc(\C=C\C(=O)NC2CCCCCC2)cc1,26.427,8,0.300,0,4.238,1.263,0.474,0,2,3.746,Inactive
4,Brc1ccc(cc1)C(=O)N\N=C\c2ccc3cccc(OCc4ccccc4)c3n2,35.370,25,0.697,0,4.612,1.124,0.850,0,4,4.476,Active
...,...,...,...,...,...,...,...,...,...,...,...,...
2732,c1cc(ccn1)c2sc(c3ccncc3)c(c4ccncc4)c2c5ccncc5,34.239,29,0.879,0,4.365,0.849,0.966,0,8,2.738,Active
2733,c1ccc(cc1)n2ncc3c2ncn4cnnc34,18.796,20,0.952,1,3.890,1.019,0.889,0,10,3.366,Inactive
2734,c1ccc2c(nc(nc2c1)c3cccnc3)n4ccnc4,23.313,22,0.917,1,4.014,0.965,0.952,0,11,2.861,Inactive
2735,c1ccc2nc(ccc2c1)c3ccncc3,19.057,17,0.944,0,3.731,1.122,1.000,0,4,2.232,Active


In [16]:
X_test = df_test.drop(["SMILES", "Class"], axis = 1)

In [17]:
y_test = df_test["Class"].map({"Active": 1, "Inactive": 0}).values

In [20]:
imputer=SimpleImputer(missing_values=np.nan,strategy='median')
imputer=imputer.fit(X_train)

In [24]:
param_grid = {
    'max_depth': [4,5,6,7,8,9,10],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 500, 1000]
}

In [26]:
rfc=RandomForestClassifier(random_state=42)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [4, 5, 6, 7, 8, 9, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 300, 500, 1000]})

In [27]:
CV_rfc.best_params_

{'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [32]:
pred=CV_rfc.predict(X_test)
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

Accuracy for Random Forest on CV data:  0.8096455973693826


In [35]:
joblib.dump(imputer, "median_imputer.joblib", compress=True)

['median_imputer.joblib']

In [36]:
joblib.dump(CV_rfc, "CV_random_forest_classifier.joblib", compress=True)

['CV_random_forest_classifier.joblib']

In [37]:
joblib.dump(CV_rfc.best_estimator_, "CV_random_forest_classifier_best_estimator.joblib", compress=True)

['CV_random_forest_classifier_best_estimator.joblib']